In [1]:
# Step 1: Import libraries and initialize
import ee
import geemap
ee.Authenticate()  # Run this only once
ee.Initialize()

# Step 2: Create map and add Bangladesh
Map = geemap.Map()
bangladesh = ee.FeatureCollection("FAO/GAUL/2015/level0") \
              .filter(ee.Filter.eq('ADM0_NAME', 'Bangladesh'))
Map.addLayer(bangladesh, {}, 'Bangladesh')
Map.centerObject(bangladesh, 7)

# Step 3: Updated Landsat collection function
def get_landsat_data(start_year, end_year):
    # Landsat 5 (Collection 2)
    if end_year <= 2011:
        collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
        # Band mapping for Collection 2
        blue, green, red, nir = 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4'
        
    # Landsat 7 (Collection 2)
    elif start_year <= 2022:
        collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
        blue, green, red, nir = 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4'
        
    # Landsat 8/9 (Collection 2)
    else:
        collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        blue, green, red, nir = 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5'
    
    # Filter by date and location
    collection = collection.filterBounds(bangladesh) \
                          .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')
    
    # Cloud masking function for Collection 2
    def mask_clouds(image):
        qa = image.select('QA_PIXEL')
        cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # Cloud bit
        return image.updateMask(cloud_mask)
    
    # Apply scaling factors for Collection 2
    def apply_scale_factors(image):
        optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        return image.addBands(optical_bands, None, True)
    
    return collection.map(mask_clouds).map(apply_scale_factors).median()

# Step 4: Get and visualize 2000-2005 data
ls_2000_2005 = get_landsat_data(2000, 2005)

# Add to map (True Color)
Map.addLayer(
    ls_2000_2005.clip(bangladesh),
    {'bands': ['SR_B3', 'SR_B2', 'SR_B1'], 'min': 0, 'max': 0.3},
    'Landsat 2000-2005 (True Color)'
)

# Step 5: Calculate NDVI
def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    return image.addBands(ndvi)

ls_2000_2005 = add_ndvi(ls_2000_2005)

# Add NDVI layer
ndvi_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}
Map.addLayer(ls_2000_2005.select('NDVI').clip(bangladesh), ndvi_params, 'NDVI 2000-2005')

# Display the map   
Map

Map(center=[23.8270519912324, 90.28837386710796], controls=(WidgetControl(options=['position', 'transparent_bg…